# ANALISI DATI DI TEST

#### Autore: Enrico Brunelli

#### Data ultima modifica: 19/11/2024

### Descrizione:
 I test effettuati sono stati condotti variando i nodi di rete che generano traffico. Tali nodi sono stati generati da lato client connettendo tre distinti moduli Quectel EVB connessi al PC MSI con Windows 11. In tal modo è stato possibile utilizzare la rete dei moduli contemporaneamente. Sono stati implementati tre client tramite il software iperf3. Lato server, sul PC Firecell che contiene il Core Network della rete 5G, sono stati configurati 3 server tramite iperf3 sono 3 distinte porte: 5201 6201 7201.
 Inoltre è stato analizzato l'andamento dei container tramite il comando *docket stats* durante la connessione di 6 distinti User Equipement rappresentati dai moduli Quectel. In tal contesto è stato anche analizzato e catturato il traffico sull'interfaccia *enp5s0* in modo da osservare l'andamento dei pacchetti durante la fase di registrazione sulla rete. 


In [22]:
# Librerie necessarie all'analisi dei dati e la relativa rappresentazione
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ntplib import NTPStats
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MultipleLocator

In [23]:
# Nomi dei file rispettivamente per Client e Server
file_client="test_19_11_2024/istanti_temporali_2024-11-19.csv"
file_server="test_19_11_2024/server_istanti_temporali_2024-11-19.csv"

df1 = pd.read_csv(file_client) #legge il file del client
df2 = pd.read_csv(file_server) #legge il file del server

# Converto le colonne del file del Client che contengono l'ntp object
print(df1)
print(df2)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp
0            TCP         0           1.732011e+09           1.732011e+09
1            TCP         0           1.732011e+09           1.732011e+09
2            TCP         0           1.732011e+09           1.732011e+09
3            TCP         0           1.732011e+09           1.732011e+09
4            TCP         0           1.732011e+09           1.732011e+09
...          ...       ...                    ...                    ...
15943        TCP         0           1.732013e+09           1.732013e+09
15944        TCP         0           1.732013e+09           1.732013e+09
15945        TCP         0           1.732013e+09           1.732013e+09
15946        TCP         0           1.732013e+09           1.732013e+09
15947        TCP         0           1.732013e+09           1.732013e+09

[15948 rows x 4 columns]
      Tipo  server_send_timestamp  server_recv_timestamp
0      TCP           1.732011e+09        

In [24]:
# Unisco i risultati ottenuti lato client con quelli ottenuti lato server

df1['server_send_timestamp']=df2['server_send_timestamp']
df1['server_recv_timestamp']=df2['server_recv_timestamp']
print(df1)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp  \
0            TCP         0           1.732011e+09           1.732011e+09   
1            TCP         0           1.732011e+09           1.732011e+09   
2            TCP         0           1.732011e+09           1.732011e+09   
3            TCP         0           1.732011e+09           1.732011e+09   
4            TCP         0           1.732011e+09           1.732011e+09   
...          ...       ...                    ...                    ...   
15943        TCP         0           1.732013e+09           1.732013e+09   
15944        TCP         0           1.732013e+09           1.732013e+09   
15945        TCP         0           1.732013e+09           1.732013e+09   
15946        TCP         0           1.732013e+09           1.732013e+09   
15947        TCP         0           1.732013e+09           1.732013e+09   

       server_send_timestamp  server_recv_timestamp  
0               1.732011e+09     

In [25]:
# Calcolo RTT e OTT 
df1['RTT']=(df1['client_recv_timestamp']-df1['server_send_timestamp'])+(df1['server_recv_timestamp']-df1['client_send_timestamp'])
df1['OTT_send']=df1['server_recv_timestamp']-df1['client_send_timestamp']
df1['OTT_recv']=df1['client_recv_timestamp']-df1['server_send_timestamp']
print(df1)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp  \
0            TCP         0           1.732011e+09           1.732011e+09   
1            TCP         0           1.732011e+09           1.732011e+09   
2            TCP         0           1.732011e+09           1.732011e+09   
3            TCP         0           1.732011e+09           1.732011e+09   
4            TCP         0           1.732011e+09           1.732011e+09   
...          ...       ...                    ...                    ...   
15943        TCP         0           1.732013e+09           1.732013e+09   
15944        TCP         0           1.732013e+09           1.732013e+09   
15945        TCP         0           1.732013e+09           1.732013e+09   
15946        TCP         0           1.732013e+09           1.732013e+09   
15947        TCP         0           1.732013e+09           1.732013e+09   

       server_send_timestamp  server_recv_timestamp       RTT  OTT_send  \
0           

In [29]:

df1['RTT']=df1[df1['RTT']<0.100]
df1['RTT']=df1[df1['RTT']>0.010]
stats = df1.groupby(['Traffico'])['RTT'].agg(['mean', 'var','std','median','count','min','max'])
print(stats.round(2))


ValueError: Columns must be same length as key